# Aaron Kohn
## Final Project; Milestone 3

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
# List of school names, numbers and district
# Extact tables from html
file = str('List of public elementary schools in New York City - Wikipedia.html')
fd = open(file, "r", encoding= 'utf8')
soup = BeautifulSoup(fd)
fd.close()
tables = soup.find_all('table', {'class' : 'wikitable'})
frame = []
for table in tables:
    header = [th.get_text().strip() for th in table.tbody.find_all('tr')[0].find_all('th')]
    rows = table.tbody.find_all('tr')[1:]
    data = [[td.get_text().strip() for td in tr.find_all('td')] for tr in rows]
    df = pd.DataFrame(data, columns= header)
    frame.append(df)
    
# Extract table titles
heads = [h3.get_text().strip() for h3 in soup.find_all('h3')]
heads = ['Manhattan'] + heads[0:13]

# Create data frame with titles as index
ps_df = pd.concat(frame, keys = heads)

# Extract necassary columns and rows
ps_df = ps_df[['SchoolNumber', 'School Name', 'Neighborhood']]
ps_df.drop('Charter schools: citywide[edit]', inplace= True)
ps_df = ps_df.reset_index()
ps_df.drop('level_1', axis= 1, inplace= True)
ps_df.rename(columns={'level_0':'Borough'}, inplace=True)
# ps_df.to_csv('NYC.csv')
# ps_df = pd.read_csv('NYC.csv')
ps_df

,Borough,SchoolNumber,School Name,Neighborhood
0,Manhattan,PS 1,Alfred F. Smith,Two Bridges
1,Manhattan,PS 11,William T. Harris,Chelsea
2,Manhattan,PS 15,Roberto Clemente,East Village
3,Manhattan,PS 18,Park Terrace,Inwood
4,Manhattan,PS 19,Asher Levy,East Village
...,...,...,...,...
772,Special Education District 75: citywide[edit],PS 231K,,
773,Special Education District 75: citywide[edit],PS 327,The Children’s School,
774,Special Education District 75: citywide[edit],PS 373 R,,
775,Special Education District 75: citywide[edit],PS 352 X,The Vida Bogart School for All Children,


In [3]:
# Switch column School Name to lower case for matching
ps_df['School Name'] = ps_df['School Name'].apply(lambda x : x.lower())
ps_df.head()

,Borough,SchoolNumber,School Name,Neighborhood
0,Manhattan,PS 1,alfred f. smith,Two Bridges
1,Manhattan,PS 11,william t. harris,Chelsea
2,Manhattan,PS 15,roberto clemente,East Village
3,Manhattan,PS 18,park terrace,Inwood
4,Manhattan,PS 19,asher levy,East Village


In [4]:
# Import school data information from multiple pages(1 page per file)
frame2 = [] # Create list to store pages
for i in range(1, 138): # Iterate through files named in order
    file = str('state ' + str(i) +'.html') # create file names
    fd = open(file, "r", encoding= 'utf8')
    soup = BeautifulSoup(fd)
    fd.close()
    table = soup.find('table', {'class' : 'ls'})
    header = [td.get_text().strip() for td in table.tbody.find_all('tr')[0].find_all('td')]
    rows = table.tbody.find_all('tr')[1:]
    data = [[td.get_text().strip() for td in tr.find_all('td')] for tr in rows]
    html_df = pd.DataFrame(data, columns= header)
    frame2.append(html_df)

In [5]:
full_html = pd.concat(frame2, ignore_index=True) # merge all pages to one frame
# full_html.to_csv('NYS.csv') # Save to csv
# full_html = pd.read_csv('NYS.csv') # Read from saved csv
full_html.head()

,Institution ID,Legal Name,Popular Name,SED Code,INST Type Code,INST Type Description,INST Sub Type Code,INST Sub Type Description,County Code,County Description,...,Grade 5,Grade 6,Grade Ungraded Elementary,Grade 7,Grade 8,Grade 9,Grade 10,Grade 11,Grade 12,Grade Ungraded Secondary
0,-800000090834,HELLENIC CLASSICAL CHARTER SCHOOL - STATEN ISLAND,HELLENIC CLASSICAL CS -STATEN ISLAND,353100861175,17,PUBLIC SCHOOLS,11,CHARTER SCHOOL,35,RICHMOND,...,,,,,,,,,,
1,-800000059005,SMILES AROUND US ACADEMY,SMILES AROUND US ACADEMY,353100995152,14,NON-PUBLIC SCHOOLS,21,INDEPENDENT,35,RICHMOND,...,Y,Y,,Y,,,,,,
2,-800000091532,"EDEN II SCHOOL FOR AUTISTIC CHILDREN, INC. LIT...",EDEN II - LITTLE MIRACLES,353100996532,14,NON-PUBLIC SCHOOLS,21,INDEPENDENT,35,RICHMOND,...,,,,,,,,,,
3,-800000091977,EDEN II SCHOOL FOR AUTISTIC CHILDREN - MANOR ROAD,EDEN II - MANOR ROAD,353100996570,14,NON-PUBLIC SCHOOLS,21,INDEPENDENT,35,RICHMOND,...,,,,,,,,,,
4,-800000041775,STATEN ISLAND ACADEMY,STATEN ISLAND ACADEMY,353100997022,14,NON-PUBLIC SCHOOLS,21,INDEPENDENT,35,RICHMOND,...,Y,Y,,Y,Y,Y,Y,Y,Y,


In [6]:
# Select desired rows
public_df = full_html[full_html['INST Sub Type Description'] == 'PUBLIC SCHOOL CITY'] # Select only City Public Schools
fiveborough = ['RICHMOND', 'QUEENS', 'KINGS', 'BRONX', 'NEW YORK']
public_df = public_df[(public_df["County Description"].isin(fiveborough))] # Select only NYC schools

public_df

,Institution ID,Legal Name,Popular Name,SED Code,INST Type Code,INST Type Description,INST Sub Type Code,INST Sub Type Description,County Code,County Description,...,Grade 5,Grade 6,Grade Ungraded Elementary,Grade 7,Grade 8,Grade 9,Grade 10,Grade 11,Grade 12,Grade Ungraded Secondary
60,-800000088504,PORT RICHMOND SCHOOL FOR VISIONARY LEARNING,PORT RICHMOND SCHOOL FOR VISIONARY,353100010068,17,PUBLIC SCHOOLS,1,PUBLIC SCHOOL CITY,35,RICHMOND,...,,,,,,,,,,
61,-800000042011,PS 69 DANIEL D TOMPKINS,PS 69 DANIEL D TOMPKINS,353100010069,17,PUBLIC SCHOOLS,1,PUBLIC SCHOOL CITY,35,RICHMOND,...,Y,,,,,,,,,
62,-800000042012,IS 72 ROCCO LAURIE,IS 72 ROCCO LAURIE,353100010072,17,PUBLIC SCHOOLS,1,PUBLIC SCHOOL CITY,35,RICHMOND,...,,Y,,Y,Y,,,,,Y
63,-800000069167,PS 74 FUTURE LEADERS ELEMENTARY SCHOOL,PS 74 FUTURE LEADERS ELEMENTARY,353100010074,17,PUBLIC SCHOOLS,1,PUBLIC SCHOOL CITY,35,RICHMOND,...,Y,,,,,,,,,
64,-800000042013,IS 75 FRANK D PAULO,IS 75 FRANK D PAULO,353100010075,17,PUBLIC SCHOOLS,1,PUBLIC SCHOOL CITY,35,RICHMOND,...,,Y,Y,Y,Y,,,,,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2735,-800000046867,PS 169 ROBERT F KENNEDY,PS 169 ROBERT F KENNEDY,307500011169,17,PUBLIC SCHOOLS,1,PUBLIC SCHOOL CITY,31,NEW YORK,...,Y,Y,Y,Y,Y,Y,,Y,Y,Y
2736,-800000046870,PS 226,PS 226,307500011226,17,PUBLIC SCHOOLS,1,PUBLIC SCHOOL CITY,31,NEW YORK,...,,Y,Y,Y,,Y,Y,,Y,Y
2737,-800000046871,HOSPITAL SCHOOLS,HOSPITAL SCHOOLS,307500011401,17,PUBLIC SCHOOLS,1,PUBLIC SCHOOL CITY,32,BRONX,...,,,Y,,,,,,,
2738,-800000046872,PS 721 MANHATTAN OCCUPATIONAL TRAINING CENTER,PS 721 MANHATTAN OCC TRNING CTR,307500011721,17,PUBLIC SCHOOLS,1,PUBLIC SCHOOL CITY,31,NEW YORK,...,,,Y,,,Y,Y,Y,Y,Y


In [7]:
# Switch column Legal Name to lower case for matching
public_df['Legal Name'] = public_df['Legal Name'].apply(lambda x : x.lower())
public_df.head()

,Institution ID,Legal Name,Popular Name,SED Code,INST Type Code,INST Type Description,INST Sub Type Code,INST Sub Type Description,County Code,County Description,...,Grade 5,Grade 6,Grade Ungraded Elementary,Grade 7,Grade 8,Grade 9,Grade 10,Grade 11,Grade 12,Grade Ungraded Secondary
60,-800000088504,port richmond school for visionary learning,PORT RICHMOND SCHOOL FOR VISIONARY,353100010068,17,PUBLIC SCHOOLS,1,PUBLIC SCHOOL CITY,35,RICHMOND,...,,,,,,,,,,
61,-800000042011,ps 69 daniel d tompkins,PS 69 DANIEL D TOMPKINS,353100010069,17,PUBLIC SCHOOLS,1,PUBLIC SCHOOL CITY,35,RICHMOND,...,Y,,,,,,,,,
62,-800000042012,is 72 rocco laurie,IS 72 ROCCO LAURIE,353100010072,17,PUBLIC SCHOOLS,1,PUBLIC SCHOOL CITY,35,RICHMOND,...,,Y,,Y,Y,,,,,Y
63,-800000069167,ps 74 future leaders elementary school,PS 74 FUTURE LEADERS ELEMENTARY,353100010074,17,PUBLIC SCHOOLS,1,PUBLIC SCHOOL CITY,35,RICHMOND,...,Y,,,,,,,,,
64,-800000042013,is 75 frank d paulo,IS 75 FRANK D PAULO,353100010075,17,PUBLIC SCHOOLS,1,PUBLIC SCHOOL CITY,35,RICHMOND,...,,Y,Y,Y,Y,,,,,Y


In [8]:
# Add column School name_num by combining SchoolNumber and School Name for matching
ps_df['School name_num'] = ps_df['SchoolNumber']+' ' + ps_df['School Name']
ps_df['School name_num'] = ps_df['School name_num'].apply(lambda x : x.lower()) # Convert to lowercase
ps_df

,Borough,SchoolNumber,School Name,Neighborhood,School name_num
0,Manhattan,PS 1,alfred f. smith,Two Bridges,ps 1 alfred f. smith
1,Manhattan,PS 11,william t. harris,Chelsea,ps 11 william t. harris
2,Manhattan,PS 15,roberto clemente,East Village,ps 15 roberto clemente
3,Manhattan,PS 18,park terrace,Inwood,ps 18 park terrace
4,Manhattan,PS 19,asher levy,East Village,ps 19 asher levy
...,...,...,...,...,...
772,Special Education District 75: citywide[edit],PS 231K,,,ps 231k
773,Special Education District 75: citywide[edit],PS 327,the children’s school,,ps 327 the children’s school
774,Special Education District 75: citywide[edit],PS 373 R,,,ps 373 r
775,Special Education District 75: citywide[edit],PS 352 X,the vida bogart school for all children,,ps 352 x the vida bogart school for all children


In [9]:
# making separate column for school number
# new = public_df["Legal Name"].str.split(" ",  expand = True) 
# public_df["School_Num"]= new[1]
# public_df.head(3)

In [10]:
from fuzzywuzzy import process, fuzz
import logging
logging.getLogger().setLevel(logging.ERROR)


In [11]:
# Use fuzzywuzzy token_sort_ratio to find matching school names retirns columns with name found and similarity ratio
actual_name = []
similarity = []
for i in ps_df['School name_num']:
        ratio = process.extract( i, public_df['Legal Name'], limit=1, scorer= fuzz.token_sort_ratio)
        actual_name.append(ratio[0][0])
        similarity.append(ratio[0][1])
ps_df['actual_name'] = pd.Series(actual_name)
ps_df['similarity'] = pd.Series(similarity)
ps_df

,Borough,SchoolNumber,School Name,Neighborhood,School name_num,actual_name,similarity
0,Manhattan,PS 1,alfred f. smith,Two Bridges,ps 1 alfred f. smith,ps 1 alfred e smith,95
1,Manhattan,PS 11,william t. harris,Chelsea,ps 11 william t. harris,ps 11 william t harris,100
2,Manhattan,PS 15,roberto clemente,East Village,ps 15 roberto clemente,ps 15 roberto clemente,100
3,Manhattan,PS 18,park terrace,Inwood,ps 18 park terrace,ps 18 park terrace,100
4,Manhattan,PS 19,asher levy,East Village,ps 19 asher levy,ps 19 asher levy,100
...,...,...,...,...,...,...,...
772,Special Education District 75: citywide[edit],PS 231K,,,ps 231k,ps 231,92
773,Special Education District 75: citywide[edit],PS 327,the children’s school,,ps 327 the children’s school,ps 372 children's school (the),96
774,Special Education District 75: citywide[edit],PS 373 R,,,ps 373 r,ps 373,86
775,Special Education District 75: citywide[edit],PS 352 X,the vida bogart school for all children,,ps 352 x the vida bogart school for all children,vida bogart school for all children (the),90


In [12]:
# merge data frames on fuzzy matched names
merged_df = ps_df.merge(public_df, left_on= 'actual_name', right_on= 'Legal Name')

In [13]:
# Drop unwanted and duplicated columns
merged_df.drop(['Institution ID', 'Popular Name', 'actual_name', 'SED Code', 'INST Type Code','INST Sub Type Code', 'Physical Address Line1', 'Physical Address Line2',
               'Physical State Code', 'Physical Zipcd5', 'Mailing Address Line1', 'Mailing Address Line2', 'Mailing City', 'Mailing State Code',
               'Mailing Zipcd5', 'CEO email', 'CEO Phone', 'CEO Phone Extension', 'Grade Organization Code'], axis = 1, inplace= True)


In [14]:
# Drop duplicate schools (based on name and county code)
merged_df = merged_df.drop_duplicates(subset= ['SchoolNumber', 'County Code'])

merged_df

,Borough,SchoolNumber,School Name,Neighborhood,School name_num,similarity,Legal Name,INST Type Description,INST Sub Type Description,County Code,...,Grade 5,Grade 6,Grade Ungraded Elementary,Grade 7,Grade 8,Grade 9,Grade 10,Grade 11,Grade 12,Grade Ungraded Secondary
0,Manhattan,PS 1,alfred f. smith,Two Bridges,ps 1 alfred f. smith,95,ps 1 alfred e smith,PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,31,...,Y,,,,,,,,,
1,Manhattan,PS 11,william t. harris,Chelsea,ps 11 william t. harris,100,ps 11 william t harris,PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,31,...,Y,,Y,,,,,,,
3,Manhattan,PS 15,roberto clemente,East Village,ps 15 roberto clemente,100,ps 15 roberto clemente,PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,31,...,Y,,Y,,,,,,,
5,Manhattan,PS 18,park terrace,Inwood,ps 18 park terrace,100,ps 18 park terrace,PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,31,...,Y,Y,,Y,Y,,,,,
7,Manhattan,PS 19,asher levy,East Village,ps 19 asher levy,100,ps 19 asher levy,PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,31,...,Y,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,Special Education District 75: citywide[edit],PS 224 Q,,,ps 224 q,86,ps 224,PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,34,...,Y,Y,Y,Y,Y,,,,,Y
778,Special Education District 75: citywide[edit],PS 231K,,,ps 231k,92,ps 231,PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,33,...,Y,Y,Y,Y,Y,,,,,Y
779,Special Education District 75: citywide[edit],PS 373 R,,,ps 373 r,86,ps 373,PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,35,...,Y,,Y,,Y,,,,,Y
780,Special Education District 75: citywide[edit],PS 352 X,the vida bogart school for all children,,ps 352 x the vida bogart school for all children,90,vida bogart school for all children (the),PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,32,...,Y,Y,Y,Y,Y,,,,,


In [15]:
# drop rows missing school number
merged_df['SchoolNumber'].replace('', np.nan, inplace= True ) # Replace empty string with nan
merged_df = merged_df[merged_df['SchoolNumber'].notna()]
merged_df

,Borough,SchoolNumber,School Name,Neighborhood,School name_num,similarity,Legal Name,INST Type Description,INST Sub Type Description,County Code,...,Grade 5,Grade 6,Grade Ungraded Elementary,Grade 7,Grade 8,Grade 9,Grade 10,Grade 11,Grade 12,Grade Ungraded Secondary
0,Manhattan,PS 1,alfred f. smith,Two Bridges,ps 1 alfred f. smith,95,ps 1 alfred e smith,PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,31,...,Y,,,,,,,,,
1,Manhattan,PS 11,william t. harris,Chelsea,ps 11 william t. harris,100,ps 11 william t harris,PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,31,...,Y,,Y,,,,,,,
3,Manhattan,PS 15,roberto clemente,East Village,ps 15 roberto clemente,100,ps 15 roberto clemente,PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,31,...,Y,,Y,,,,,,,
5,Manhattan,PS 18,park terrace,Inwood,ps 18 park terrace,100,ps 18 park terrace,PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,31,...,Y,Y,,Y,Y,,,,,
7,Manhattan,PS 19,asher levy,East Village,ps 19 asher levy,100,ps 19 asher levy,PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,31,...,Y,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,Special Education District 75: citywide[edit],PS 224 Q,,,ps 224 q,86,ps 224,PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,34,...,Y,Y,Y,Y,Y,,,,,Y
778,Special Education District 75: citywide[edit],PS 231K,,,ps 231k,92,ps 231,PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,33,...,Y,Y,Y,Y,Y,,,,,Y
779,Special Education District 75: citywide[edit],PS 373 R,,,ps 373 r,86,ps 373,PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,35,...,Y,,Y,,Y,,,,,Y
780,Special Education District 75: citywide[edit],PS 352 X,the vida bogart school for all children,,ps 352 x the vida bogart school for all children,90,vida bogart school for all children (the),PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,32,...,Y,Y,Y,Y,Y,,,,,


In [16]:
# Fill true/false data with Y/N may require replacement with boolean values
merged_df.iloc[:, -18:-1] = merged_df.iloc[:, -18:-1].replace('', 'N') # Replace empty string with 'N'
merged_df

C:\Users\aarme\anaconda3.2\lib\site-packages\pandas\core\indexing.py:1754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)


,Borough,SchoolNumber,School Name,Neighborhood,School name_num,similarity,Legal Name,INST Type Description,INST Sub Type Description,County Code,...,Grade 5,Grade 6,Grade Ungraded Elementary,Grade 7,Grade 8,Grade 9,Grade 10,Grade 11,Grade 12,Grade Ungraded Secondary
0,Manhattan,PS 1,alfred f. smith,Two Bridges,ps 1 alfred f. smith,95,ps 1 alfred e smith,PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,31,...,Y,N,N,N,N,N,N,N,N,
1,Manhattan,PS 11,william t. harris,Chelsea,ps 11 william t. harris,100,ps 11 william t harris,PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,31,...,Y,N,Y,N,N,N,N,N,N,
3,Manhattan,PS 15,roberto clemente,East Village,ps 15 roberto clemente,100,ps 15 roberto clemente,PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,31,...,Y,N,Y,N,N,N,N,N,N,
5,Manhattan,PS 18,park terrace,Inwood,ps 18 park terrace,100,ps 18 park terrace,PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,31,...,Y,Y,N,Y,Y,N,N,N,N,
7,Manhattan,PS 19,asher levy,East Village,ps 19 asher levy,100,ps 19 asher levy,PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,31,...,Y,N,N,N,N,N,N,N,N,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,Special Education District 75: citywide[edit],PS 224 Q,,,ps 224 q,86,ps 224,PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,34,...,Y,Y,Y,Y,Y,N,N,N,N,Y
778,Special Education District 75: citywide[edit],PS 231K,,,ps 231k,92,ps 231,PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,33,...,Y,Y,Y,Y,Y,N,N,N,N,Y
779,Special Education District 75: citywide[edit],PS 373 R,,,ps 373 r,86,ps 373,PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,35,...,Y,N,Y,N,Y,N,N,N,N,Y
780,Special Education District 75: citywide[edit],PS 352 X,the vida bogart school for all children,,ps 352 x the vida bogart school for all children,90,vida bogart school for all children (the),PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,32,...,Y,Y,Y,Y,Y,N,N,N,N,


In [17]:
# Replace empty strings with nan for remaining columns
merged_df.replace('', np.nan, inplace= True)
merged_df

C:\Users\aarme\anaconda3.2\lib\site-packages\pandas\core\frame.py:4524: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


,Borough,SchoolNumber,School Name,Neighborhood,School name_num,similarity,Legal Name,INST Type Description,INST Sub Type Description,County Code,...,Grade 5,Grade 6,Grade Ungraded Elementary,Grade 7,Grade 8,Grade 9,Grade 10,Grade 11,Grade 12,Grade Ungraded Secondary
0,Manhattan,PS 1,alfred f. smith,Two Bridges,ps 1 alfred f. smith,95,ps 1 alfred e smith,PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,31,...,Y,N,N,N,N,N,N,N,N,NaN
1,Manhattan,PS 11,william t. harris,Chelsea,ps 11 william t. harris,100,ps 11 william t harris,PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,31,...,Y,N,Y,N,N,N,N,N,N,NaN
3,Manhattan,PS 15,roberto clemente,East Village,ps 15 roberto clemente,100,ps 15 roberto clemente,PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,31,...,Y,N,Y,N,N,N,N,N,N,NaN
5,Manhattan,PS 18,park terrace,Inwood,ps 18 park terrace,100,ps 18 park terrace,PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,31,...,Y,Y,N,Y,Y,N,N,N,N,NaN
7,Manhattan,PS 19,asher levy,East Village,ps 19 asher levy,100,ps 19 asher levy,PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,31,...,Y,N,N,N,N,N,N,N,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,Special Education District 75: citywide[edit],PS 224 Q,NaN,NaN,ps 224 q,86,ps 224,PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,34,...,Y,Y,Y,Y,Y,N,N,N,N,Y
778,Special Education District 75: citywide[edit],PS 231K,NaN,NaN,ps 231k,92,ps 231,PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,33,...,Y,Y,Y,Y,Y,N,N,N,N,Y
779,Special Education District 75: citywide[edit],PS 373 R,NaN,NaN,ps 373 r,86,ps 373,PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,35,...,Y,N,Y,N,Y,N,N,N,N,Y
780,Special Education District 75: citywide[edit],PS 352 X,the vida bogart school for all children,NaN,ps 352 x the vida bogart school for all children,90,vida bogart school for all children (the),PUBLIC SCHOOLS,PUBLIC SCHOOL CITY,32,...,Y,Y,Y,Y,Y,N,N,N,N,NaN


In [18]:
merged_df.to_csv('merged_df.csv')